Copyright 2024 Google, LLC. This software is provided as-is,
without warranty or representation for any use or purpose. Your
use of it is subject to your agreement with Google.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

   http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

# How to use Function Calling with Gemini

This notebook outlines how to interact with Vertex AI's Gemini models to call external API's using Function Calling. More info can be found at https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/function-calling

## Prepare the python development environment

First, let's identify any project specific variables to customize this notebook to your GCP environment. Change YOUR_PROJECT_ID with your own GCP project ID.

In [ ]:
project_id = "YOUR_PROJECT_ID"
location = "global"
region = "us-central1"
key_file = "key.txt"

Install any needed python modules from our requirements.txt file. Most Vertex Workbench environments include all the packages we'll be using, but if you are using an external Jupyter Notebook or require any additional packages for your own needs, you can simply add them to the included requirements.txt file an run the folloiwng commands.

In [ ]:
#pip install -r requirements.txt

Now we will import all required modules. For our purpose, we will be utilizing the following:

- requests - This module will allow us to interact directly with external REST API's. 
- FunctionDeclaration - Used to define the function to be called by the model

In [ ]:
import vertexai
from vertexai.generative_models import GenerativeModel, GenerationConfig, Part, Tool, ChatSession, FunctionDeclaration
import vertexai.preview.generative_models as generative_models
from vertexai.preview.generative_models import grounding, ToolConfig
import requests
from pathlib import Path

## Verify the API key for weatherapi.com

You will need to add your own API key to the key.txt file in order to access the weatherapi endpoint. Please visit https://www.weatherapi.com/ to sign up for a free account and generate your own API key. Once you have your own API key, replace all the text in the key.txt file with your new key.

In this example we are using a simple file to store the API key, but you will want to use something like GCP Secret Manager for a production environment.

In [ ]:
# Verify the API key has been added to the key.txt file
weather_api_key = Path(key_file).read_text()

if weather_api_key.startswith('Replace all the text') == True:
        print('API key has not been added')
        raise Exception(f'The {key_file} file does not contain a valid API key. Please visit https://www.weatherapi.com/ to sign up for a free account and generate your own API key.')
else:
        print(f'The {key_file} file appears to contain a valid API key.')

## Define the Gemini Functions

Next we will define the functions to be called by the gemini model. This function will be used to determine the current time based on a specified location provided by the user.

In [ ]:
get_time_func = FunctionDeclaration(
    name="get_time_func",
    description="Get the current time in a given location. Include the area and location, for example 'area: America/New_York', 'Asia/Dubai' and 'Africa/Cairo'",
    parameters={
        "type": "object",
        "properties": {"location": {"type": "string", "description": "Location"}},
    },
)

Next we'll create a second function to get the current weather

In [ ]:
get_weather_func = FunctionDeclaration(
    name="get_weather_func",
    description="Get the current weather in a given location, for example 'Chicago, New_York, London. Replace any spaces in the name with an underscore, such as New York should be New_York.",
    parameters={
        "type": "object",
        "properties": {"location": {"type": "string", "description": "Location"}},
    },
)

Define a tool to attach the functions to Gemini

In [ ]:
example_tool = Tool(
    function_declarations=[get_time_func, get_weather_func],
)

Initialize the mode, specifying the "example_tool" variable linked to the get_time_func and get_weather_func functions

In [ ]:
model = GenerativeModel(
    "gemini-1.5-pro-002",
    generation_config=GenerationConfig(temperature=0),
    tools=[example_tool],
    tool_config=ToolConfig(
        function_calling_config=ToolConfig.FunctionCallingConfig(
            # ANY mode forces the model to predict a function call
            mode=ToolConfig.FunctionCallingConfig.Mode.AUTO,
            # Allowed functions to call when the mode is ANY. If empty, any one of
            # the provided functions are called.
            #allowed_function_names=["get_time_func"],
        )
    )
)
chat = model.start_chat()

## Define a function for determining the function call being used

By defining a function in python to determine the function call helps keep your code base clean and alloww for easily adding additional function calls in the future. We need to determine which function call is being used to ensure the appropriate parameters are being passed to the API endpoint. As you can see below, the get_weather_func function calls a different backend API and uses different parameters than the get_time_func function.

In [ ]:
def call_api(function_name, response, api_key=None):
  """
  Calls an external API based on the provided function name.

  Args:
    function_name: The name of the function to call.
    response: The response object containing function call arguments.
    api_key: Optional argument to pass the API key for weather API (required for get_weather_func).

  Returns:
    A response object with the API response text.
  """
  params = {}

  if function_name == 'get_time_func':
    for key, value in response.candidates[0].content.parts[0].function_call.args.items():
      params[key] = value
    url = f"https://worldtimeapi.org/api/timezone/{params['location']}"

  elif function_name == 'get_weather_func':
    # Get the appropriate API key info for the weather API. We are using a simple file for this example, but you will want to use something like GCP Secret Manager for a production environment
    weather_api_key = Path(api_key).read_text()
    
    for key, value in response.candidates[0].content.parts[0].function_call.args.items():
      params[key] = value
    url = f"http://api.weatherapi.com/v1/current.json?key={weather_api_key}&q={params['location']}&aqi=no"

  else:
    raise ValueError(f"Invalid function name: {function_name}")

  api_response = requests.get(url, params=params)

  # You can optionally include header information as displayed below if required by the api
  # api_response = requests.get(url, headers={'X-Api-Key': api_key}, params=params)

  # Construct and return the response for the chatbot
  response = chat.send_message(
      Part.from_function_response(
          name=function_name,
          response={
              "content": api_response.text,
          },
      ),
  )

  return response


## Submit a prompt, call the function and return the response

In this example, we're using the external worldtimeapi.org api to find the current time in a specific timezone based on a specified area. The supported areas can be listed using the following command 

In [ ]:
response = requests.get('http://worldtimeapi.org/api/timezone')
supported_areas = response.json()

for area in supported_areas:
    print(area)

Define a request to get the current time in Chicago

In [ ]:
prompt = "What time is it in San Francisco?"

Submit the prompt and print the response

In [ ]:
response = chat.send_message(prompt)

#--- Uncomment to see the full response structure
#print(response.candidates[0].content.parts[0])

#-- If the response includes the "function_call" attribute, capture the function name, call the external API, and return the response.
if response.candidates[0].content.parts[0].function_call:
    function_name = response.candidates[0].content.parts[0].function_call.name
    response = call_api(function_name, response)


In [ ]:
#--- Print the text section of the response which includes the current time.
print(response.text)

Now let's use the get_weather_func function to get the current weather condition. Notice how we are not specifying the location. Instead we are using the existing context of our chat session to infer the location.

In [ ]:
prompt = "What's the weather like there?"

In [ ]:
response = chat.send_message(prompt)

#print(response.candidates[0].content.parts[0])

if response.candidates[0].content.parts[0].function_call:
    function_name = response.candidates[0].content.parts[0].function_call.name
    response = call_api(function_name, response, key_file)

In [ ]:
print(response.text)

Now let's ask the model a simple question that will not use either of the function calls, but is still based on the context of this conversation.

In [ ]:
prompt = "What color is the sky?"

In [ ]:
response = chat.send_message(prompt)

if response.candidates[0].content.parts[0].function_call:
    function_name = response.candidates[0].content.parts[0].function_call.name
    response = call_api(function_name, response)

In [ ]:
print(response.text)

Lastly we will ask about an unrelated topic to verify the model is not solely bound to the grounding data provided by the functions. 

In [ ]:
prompt = "Why do thunderstorms form?"

In [ ]:
response = chat.send_message(prompt)

if response.candidates[0].content.parts[0].function_call:
    function_name = response.candidates[0].content.parts[0].function_call.name
    response = call_api(function_name, response)

In [ ]:
print(response.text)